In [ ]:
import crewai
from crewai import Agent ,LLM ,Task
import os


In [ ]:
llm = LLM(
    model="openrouter/meituan/longcat-flash-chat:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

In [ ]:
import csv
from datetime import datetime
from crewai import Agent, Task, Crew
import os


def create_gesture_agent_and_task(hand_sign: str):
    if hand_sign == "hello sign":
        user_prompt = """Generate unique English sentences used to greet the home assistant or start a new interaction.
Include many different tones: friendly, sleepy, excited, casual, formal, funny, impatient, etc.
Examples: "Hey assistant!", "Good morning buddy", "Wake up!", "Hi there, let's go", "Yo, you listening?" etc."""
        
    elif hand_sign == "good sign":
        user_prompt = """Generate unique English sentences expressing confirmation, satisfaction, or approval of a smart home action (locking doors, setting AC, turning off lights, vacuum starting).
Examples: "Perfect, doors are locked!", "Nice one!", "Love it, lights are off", "Yes! AC is on now", "Great job, vacuum started", "Exactly what I wanted" etc."""
        
    elif hand_sign == "pointing":
        user_prompt = """Generate unique English sentences asking the home assistant about the location of common household objects (remote, phone, keys, glasses, wallet) or rooms/devices (TV, bedroom, thermostat).
Use polite, confused, urgent, casual tones.
Examples: "Where are my keys?", "Have you seen the remote?", "Phone??", "Which room is the thermostat in?", "Where did I leave my glasses?" etc."""
    full_prompt = f"""
You are generating training data for a gesture-controlled smart home assistant.

Gesture: **{hand_sign}**

{user_prompt}

Generate **exactly 400 unique English sentences**.
Rules:
- One sentence per line
- No numbering
- No quotes
- No explanations
- Never repeat any sentence
- Make them natural and varied
"""

    agent = Agent(
        role=f"{hand_sign.title()} Gesture Specialist",
        goal=f"Generate 400 unique sentences for the '{hand_sign}' gesture",
        backstory="You are an expert in natural language for smart home voice assistants.",
        allow_delegation=False,
        verbose=False,
        llm=llm,         
        temperature=0.75,
    )

    task = Task(
        description=full_prompt,
        expected_output="Exactly 400 sentences, one per line",
        agent=agent,
    )

    return agent, task

def run_and_save(hand_sign: str):
    print(f"\nGenerating 400 sentences for → {hand_sign.upper()}")

    agent, task = create_gesture_agent_and_task(hand_sign)
    crew = Crew(agents=[agent], tasks=[task], verbose=False)

    result = crew.kickoff()
    text = result.raw if hasattr(result, "raw") else str(result)

    # Clean and collect sentences
    sentences = [line.strip() for line in text.split("\n") if line.strip() and len(line.strip()) > 5]

    # Force exactly 400
    if len(sentences) > 400:
        sentences = sentences[:400]
    while len(sentences) < 400:
        sentences.append(f"Additional {hand_sign} example {len(sentences)+1}")

    # Save CSV
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    filename = f"gesture_{hand_sign.replace(' ', '_')}_{timestamp}.csv"

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["sentence", "label"])
        for s in sentences:
            writer.writerow([s, hand_sign])

    print(f"SUCCESS → {filename} ({len(sentences)} sentences saved)")


if __name__ == "__main__":
    import time
    run_and_save("hello sign")
    time.sleep(30)
    run_and_save("good sign")
    time.sleep(30)
    run_and_save("pointing")

    print("\nAll 3 datasets complete! (1200 high-quality labeled sentences)")
    print("Files created in your current folder.")


Generating 400 sentences for → POINTING
SUCCESS → gesture_pointing_2025-11-17_19-08.csv (400 sentences saved)

All 3 datasets complete! (1200 high-quality labeled sentences)
Files created in your current folder.


In [18]:
import pandas as pd

# Step 1 — Load your CSV files
df1 = pd.read_csv("gesture_good_sign_2025-11-17_19-01.csv")
df2 = pd.read_csv("gesture_hello_sign_2025-11-17_18-44.csv")
df3 = pd.read_csv("gesture_pointing_2025-11-17_19-08.csv")

df_all = pd.concat([df1, df2, df3], ignore_index=True)

df_all = df_all.sample(frac=1, random_state=42).reset_index(drop=True)

df_all.to_csv("final_data.csv", index=False)

print("Done! Final shape:", df_all.shape)

Done! Final shape: (1200, 2)
